In [1]:
# !pip3 install gpt-2-simple

In [2]:
from gpt_2_simple.src import model as gpt2_model, encoder
import tensorflow as tf
import numpy as np
import json

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
params = '345m-hparams.json'

In [4]:
hparams = gpt2_model.default_hparams()
with open(params) as f:
    hparams.override_from_dict(json.load(f))

with open('encoder.json', 'r') as f:
    en = json.load(f)
with open('vocab.bpe', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
    
bpe_merges = [
    tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]
]
enc_malay = encoder.Encoder(encoder=en, bpe_merges=bpe_merges)

In [5]:
batch_size = 2
maxlen = 1024
num_cpu_threads = 1

def _decode_record(record, name_to_features):
    """Decodes a record to a TensorFlow example."""
    example = tf.parse_single_example(record, name_to_features)
    for name in list(example.keys()):
        t = example[name]
        if t.dtype == tf.int64:
            t = tf.to_int32(t)
        example[name] = t

    return example

name_to_features = {
    'input_ids': tf.FixedLenFeature([1024], tf.int64)
}
d = tf.data.TFRecordDataset(['gs://mesolitica-tpu-general/gpt2-testset/karangan.tfrecord'])
d = d.apply(
    tf.contrib.data.map_and_batch(
        lambda record: _decode_record(record, name_to_features),
        batch_size=batch_size,
        num_parallel_batches=num_cpu_threads,
        drop_remainder=True,
    )
)
d = d.make_one_shot_iterator().get_next()
d

Instructions for updating:
Use `tf.data.experimental.map_and_batch(...)`.
Instructions for updating:
Use `tf.data.Dataset.map(map_func, num_parallel_calls)` followed by `tf.data.Dataset.batch(batch_size, drop_remainder)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'input_ids': <tf.Tensor 'IteratorGetNext:0' shape=(2, 1024) dtype=int32>}

In [6]:
input_ids = d['input_ids']
output = gpt2_model.model(hparams=hparams, X=input_ids)
loss = tf.reduce_mean(
    input_tensor=tf.nn.sparse_softmax_cross_entropy_with_logits(
        labels=input_ids[:, 1:], logits=output['logits'][:, :-1]
    )
)
loss

<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [8]:
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'gs://mesolitica-tpu-general/gpt2-345m/model.ckpt-380000')

INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/gpt2-345m/model.ckpt-380000


In [9]:
losses = []
while True:
    try:
        l = sess.run(loss)
        print(l)
        losses.append(l)
    except Exception as e:
        print(e)
        break

1.4651937
1.5307676
1.697892
1.786686
1.6585062
1.6971546
1.7870355
1.7471293
1.8374413
1.7359962
1.8380796
2.0093484
2.0809877
1.815367
1.9282199
1.9737444
1.8878374
1.8804559
1.9029567
1.7875715
1.7148384
1.6547915
1.6174741
1.7324549
1.9313315
2.3376591
1.966411
1.9035656
1.8528032
1.9560769
1.8129612
1.6205331
1.8289036
2.0290244
1.9515997
1.7667112
1.6625499
1.5636381
1.6550319
1.7781181
1.7819471
End of sequence
	 [[node IteratorGetNext (defined at /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'IteratorGetNext':
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/dist-packages/traitlets/config/application.py", line 664, in launch_

In [11]:
import numpy as np

np.exp(np.mean(losses))

6.1040115